# Étape 2 — Nettoyage et préparation des données

Cette étape vise à corriger les problèmes de qualité identifiés lors de l’audit
notamment :
- les types de variables incorrects.
- les valeurs manquantes.
- la cohérence minimale des données.

Les données nettoyées seront sauvegardées dans le dossier `data/processed/`
afin d’être utilisées lors des analyses et des jointures ultérieures.


In [ ]:
# Import des libraries
import pandas as pd
import numpy as np


# Chargement des données brutes

In [ ]:
customers = pd.read_csv("../data/raw/customers.csv")
order_lines = pd.read_csv("../data/raw/order_lines.csv")
products = pd.read_csv("../data/raw/products.csv")


# Rappel des structures

In [ ]:
customers.info()


In [ ]:
order_lines.info()

In [ ]:
products.info()

## Nettoyage du fichier customers

Les principales actions réalisées sont :
- conversion de la variable `age` en numérique,
- conversion de `signup_date` en date,
- traitement des valeurs manquantes pour `gender` et `city`.


In [ ]:
# Types
customers["age"] = pd.to_numeric(customers["age"], errors="coerce")
customers["signup_date"] = pd.to_datetime(customers["signup_date"], errors="coerce")

# Valeurs manquantes
customers["gender"] = customers["gender"].fillna("Unknown")
customers["city"] = customers["city"].fillna("Unknown")

# Doublons
customers = customers.drop_duplicates(subset="customer_id")


In [ ]:
customers

In [ ]:
#customers.info()
customers.isna().sum()


## Nettoyage du fichier order_lines

Les actions principales incluent :
- conversion de la variable `order_date` en format `datetime`
- conversion de la variable `discount_product` en numérique
- traitement des valeurs manquantes
- gestion des doublons
- cohérence des anomalies
- préparation des données pour les contrôles de cohérence ultérieurs.


**Taille avant nettoyage**

In [ ]:
print("Taille initiale :", order_lines.shape)


**Types**

In [ ]:
# Order_date en datetime
order_lines["order_date"] = pd.to_datetime(order_lines["order_date"], errors="coerce")

In [ ]:
# discount_product en numerique
order_lines["discount_pct"] = pd.to_numeric(order_lines["discount_pct"], errors="coerce")

# Valeurs manquantes

**Justification du choix de la méthode d’imputation (Coefficient d'asymétrie)**

Avant d’imputer les valeurs manquantes des variables numériques, il est nécessaire
d’examiner la forme de leur distribution.

Le coefficient d’asymétrie mesure le degré de dissymétrie d’une distribution :
- un coefficient d’asymétrie proche de 0 indique une distribution approximativement symétrique.
- un coefficient d’asymétrie positive indique une asymétrie à droite.
- un coefficient d’asymétrie négative indique une asymétrie à gauche.

Règle de décision :
- si la distribution est approximativement symétrique (|skewness| < 0.5), 
  nous allons imputé les valeurs manquantes par la moyenne.
- si la distribution est asymétrique (|skewness| ≥ 0.5), nous allons imputé 
  les valeurs manquantes par la médiane.

Pour la variable catégorielle (`city`), on remplace les valeurs maquantes
par la modalité `"Unknown"`.

**Calcul du coefficient d'asymétrie** 

Variables concernées 
- `delivery_days`
- `review_score`

In [ ]:
# Calcul du coefficient d’asymétrie (skewness) pour les variables concernées

skew_delivery = order_lines["delivery_days"].skew()
skew_review = order_lines["review_score"].skew()

print("Coefficient d’asymétrie de delivery_days :", abs(skew_delivery))
print("Coefficient d’asymétrie de review_score :", abs(skew_review))


**Imputation**
- Les valeurs manquantes de la variable `delivery_days` seront imputées par la médiane
- Les valeurs manquantes de la variable `review_score` seront imputées par la moyenne
- Les valeurs manquantes de la variable `city` seront imputées par la modalité `"Unknown"`

In [ ]:
# Imputation des valeurs manquantes

order_lines["delivery_days"] = order_lines["delivery_days"].fillna(order_lines["delivery_days"].median())
order_lines["review_score"] = order_lines["review_score"].fillna(order_lines["review_score"].mean())
order_lines["city"] = order_lines["city"].fillna("Unknown")


**Doublons**

In [ ]:
order_lines = order_lines.drop_duplicates(subset=["order_id", "product_id"])

**Anomalies**

In [ ]:
# Anomalies détectées avant nettoyage

print("Quantités <= 0 :", (order_lines["quantity"] <= 0).sum())
print("delivery_days > 30 :", (order_lines["delivery_days"] > 30).sum())

**Vérification de cohérences**

In [ ]:
# Vérification de la Coherence des montants

order_lines["net_calc"] = (
    order_lines["gross_amount"] * (1 - order_lines["discount_pct"])
)

order_lines["amount_diff"] = order_lines["net_amount"] - order_lines["net_calc"]

print("Lignes incohérentes (|diff| > 0.01) :",
      (order_lines["amount_diff"].abs() > 0.01).sum())


**Nettoyage**

In [ ]:

order_lines_clean = order_lines.copy()

order_lines_clean = order_lines_clean[order_lines_clean["quantity"] > 0]               # Quantités non valides
order_lines_clean = order_lines_clean[order_lines_clean["delivery_days"] <= 30]        # Délai de livraison excessif
order_lines_clean = order_lines_clean[order_lines_clean["amount_diff"].abs() <= 0.01]


**Taille après nettoyage**

In [ ]:
print("Taille après nettoyage :", order_lines_clean.shape)
print("Lignes supprimées :", order_lines.shape[0] - order_lines_clean.shape[0])

**Aperçu**

In [ ]:

order_lines_clean.head()


# Fichier Products
- Déjà propres
- Pas de valeurs manquantes
- Pas de doublons attendus

In [ ]:
products = products.drop_duplicates(subset="product_id")

**Export des jeux de données vers le dossier data/processed**

In [ ]:
# Sauvegarde du fichier customers nettoyé
customers.to_csv("../data/processed/customers_clean.csv", index=False)


In [ ]:
# Sauvegarde du fichier order_lines nettoyé
order_lines_clean.to_csv("../data/processed/order_lines_clean.csv", index=False)

In [ ]:
# Sauvegarde du fichier products 
products.to_csv("../data/processed/products_clean.csv", index=False)


## Règles de nettoyage appliquées

**Règle 1 — Correction des types de variables**  
Les variables représentant des dates (`signup_date`, `order_date`) ont été converties
au format datetime. Les variables numériques (`age`, `discount_pct`) ont été converties
en type numérique, avec transformation des valeurs non valides (ex. `"unknown"`) en
valeurs manquantes.

**Règle 2 — Traitement des valeurs manquantes**  
Les valeurs manquantes ont été traitées selon leur nature : imputation par la médiane
pour la variable numérique `delivery_days` et imputation par la moyenne pour la variable 
numérique `review_score` afin de limiter l’effet des valeurs extrêmes 
et remplacement par la modalité `"Unknown"` pour les variables
catégorielles (`gender`, `city`).

**Règle 3 — Gestion des doublons**  
Les doublons ont été détectés sur la base des clés métiers pertinentes
(`customer_id` pour les clients, `order_id` et `product_id` pour les ventes) et supprimés
afin d’éviter tout double comptage dans les analyses.

**Règle 4 — Détection et suppression des anomalies métiers**  
Les observations présentant des quantités nulles ou négatives (`quantity ≤ 0`) ainsi que
des délais de livraison excessifs (`delivery_days > 30`) ont été supprimées, ces valeurs
étant jugées non réalistes d’un point de vue métier.

**Règle 5 — Contrôle de cohérence des montants financiers**  
La cohérence entre le montant brut, la remise et le montant net a été vérifiée selon la
relation : `net_amount ≈ gross_amount × (1 − discount_pct)`. Les lignes pour lesquelles
l’écart dépassait une tolérance de 0,01 (liée aux arrondis) ont été exclues du jeu de données
final.
